In [1]:
import requests
import string
from pprint import pprint
import re
import os
import tempfile
from bs4 import BeautifulSoup
from collections import Counter, OrderedDict
from operator import itemgetter
import numpy as np
hookRegex = re.compile(r'\[.*\]')
tireRegex = re.compile(r' -')
chars_to_remove = string.punctuation.replace('-', '') + '—„“»«'
print(chars_to_remove)
api_url = 'http://api.genius.com'
base_url = 'http://genius.com'
headers = {'Authorization': 'Bearer ' + open('./.bearer_token.txt', 'r').read().rstrip('\n')}

!"#$%&'()*+,./:;<=>?@[\]^_`{|}~—„“»«


In [26]:
def remove_unnecessary(textfile):
    return tireRegex.sub('', hookRegex.sub('',textfile).translate(str.maketrans('','',chars_to_remove)))

class Artist(object):
    
    def __init__(self, name):
        self.name = name
        
    artist_id = ''
    songlist = []
    
    def get_artist_id(self):
        print("Finding the ID of an artist " + self.name)
        search_url = api_url + '/search'
        data = {'q': self.name}
        response = requests.get(search_url, params=data, headers=headers)
        self.artist_id = response.json()['response']['hits'][0]['result']['primary_artist']['api_path']
        print("ID of an artist " + self.name + " is " + self.artist_id[9:])
        return
    #artists/143013/songs?sort=popularity&per_page=50
    #artists/143013/songs?sort=popularity&per_page=50&page=3
    def get_songlist(self):
        list_of_songs = []
        print("Getting the list of songs")
        for page in range(1,4):
            res = requests.get(api_url + self.artist_id + '/songs?sort=popularity&per_page=50&page=' + str(page), headers=headers)
            print("Going through page " + str(page))
            for song in res.json()['response']['songs']:
#                 print(song['full_title'])
                if ("Ft." in song['full_title']) | ("Remix" in song['full_title']):
                    print(song['full_title'] + " is a feat, skipping")
                else:
                    song_url = song['url']
                    list_of_songs.append(song_url)
        self.songlist = list_of_songs
        return
    
    
    def write_lyrics_from_path(self):
        with open("./lyrics/" + self.name.replace(' ', '_') + '_lyrics.txt', 'w') as lyricsfile:
            for song in self.songlist:
                print(str(self.songlist.index(song)) + " Now scraping " + song[19:])
                page_url = song
                res = requests.get(page_url, headers=headers)
                res.text
                page = requests.get(page_url)
                html = BeautifulSoup(page.text, 'html.parser')
                lyrics = html.find(class_='lyrics').get_text()
                lyricsfile.write(remove_unnecessary(lyrics.lower()))
        return
    
    def perform_task(self, write_id=False):
        self.get_artist_id()
        self.get_songlist()
        self.write_lyrics_from_path()
        if write_id:
            with open('./artists_id_list.txt', 'r+') as textfile:
                if self.artist_id in textfile.read():
                    pass
                else:
                    textfile.write(textfile.read() + '\n' + self.name + "  -  " + self.artist_id)
        return


In [ ]:
%%time
artists_list = ['oxxxymiron', "maculatura", "Каспийский груз", 'guf', 'Грязный рамирес', "Слава КПСС", "ЛСП", "Скриптонит", "Каста", "Кровосток", "Эджей", "ATL", "ноггано", "Big russian boss", "Bumble Beezy", "Птаха", 'pharaoh', "макс корж", "Тимати", "T-Fest", "rickey f", "SD aka Sadist", "Хаски", "Фэйс"]
for artist in artists_list:
    the_artist = Artist(artist)
    the_artist.perform_task(write_id=True)

Finding the ID of an artist oxxxymiron
ID of an artist oxxxymiron is 24422
Getting the list of songs
Going through page 1
12/04/15: Oxxxymiron vs. Johnyboy by Versus Battle (Ft. Johnyboy & Oxxxymiron) is a feat, skipping
Fata Morgana by Markul (Ft. Oxxxymiron) is a feat, skipping
Безумие (Madness) [Remix] by ЛСП (LSP) (Ft. Oxxxymiron) is a feat, skipping
Пора возвращаться домой (PVD) by БИ-2 (BI-2) (Ft. Oxxxymiron) is a feat, skipping
13/08/2017: Oxxxymiron vs. Слава КПСС by Versus Battle (Ft. Oxxxymiron & Слава КПСС (Slava KPSS)) is a feat, skipping
Больше Бена (Bolshe Bena) by Oxxxymiron (Ft. Охра (Ohra)) is a feat, skipping
Darkside by Oxxxymiron (Ft. MadChild) is a feat, skipping
Imperial by Porchy (Ft. ЛСП (LSP) & Oxxxymiron) is a feat, skipping
Бездыханным (Breathless) by Жак-Энтони (Jacques-Anthony) (Ft. Oxxxymiron) is a feat, skipping
Машина Прогресса (Progress Machine) by Ка-тет (Ka-tet) (Ft. Oxxxymiron) is a feat, skipping
Привет со дна  (Hello from the bottom) by Oxxxymiron 

Going through page 2
Going through page 3
0 Now scraping Kg-doyedesh-pishi-lyrics
1 Now scraping Kg-shugaman-lyrics
2 Now scraping Kg-tuvn-lyrics
3 Now scraping Kg-coco-lyrics
4 Now scraping Kg-tak-nevospitanno-lyrics
Finding the ID of an artist guf
ID of an artist guf is 34707
Getting the list of songs
Going through page 1
Поколение (Generation) by Тимати (Timati) (Ft. Гуф (Guf)) is a feat, skipping
18/02/2018: Guf vs Птаха by Versus Battle (Ft. Гуф (Guf) & Птаха (Ptaha)) is a feat, skipping
Вживую (Live) by Гуф (Guf) (Ft. Rigos) is a feat, skipping
Моя игра (My game) by Баста (Basta) (Ft. Гуф (Guf)) is a feat, skipping
Город дорог (Gorod Dorog) by Centr (Ft. Баста (Basta)) is a feat, skipping
Ориджинал Ба (Original Ba) by Гуф (Guf) (Ft. Original Ba XX) is a feat, skipping
На таран (Na taran) by Гуф (Guf) (Ft. Centr) is a feat, skipping
Домашняя (Domashnaya) by GUSLI (Ft. Rigos) is a feat, skipping
Нет конфликта (Net konflikta) by Кравц (Kravz) (Ft. Гуф (Guf)) is a feat, skipping
Скаж

Going through page 1
Sara Jay by SidxRam (Ft. Redo) is a feat, skipping
Рак Ушка (Rak Ushka) by SidxRam (Ft. Elias Fogg) is a feat, skipping
Going through page 2
Going through page 3
0 Now scraping Sidxram-jean-grey-lyrics
1 Now scraping Sidxram-toxin-lyrics
2 Now scraping Sidxram-ramsing-lyrics
3 Now scraping Sidxram-mereana-mordegard-lyrics
4 Now scraping Sidxram-pandemonium-lyrics
5 Now scraping Sidxram-gang-bang-lyrics
6 Now scraping Sidxram-klinch-lyrics
7 Now scraping Sidxram-dribling-lyrics
8 Now scraping Sidxram-sars-lyrics
9 Now scraping Sidxram-amadeus-lyrics
10 Now scraping Sidxram-oz-lyrics
11 Now scraping Sidxram-omen-lyrics
12 Now scraping Sidxram-reptile-lyrics
13 Now scraping Sidxram-apocalipsis-lyrics
14 Now scraping Sidxram-intrigi-iglesiasa-lyrics
15 Now scraping Sidxram-pako-vibe-lyrics
16 Now scraping Sidxram-raybax-lyrics
17 Now scraping Sidxram-ledger-lyrics
18 Now scraping Sidxram-narcosis-lyrics
19 Now scraping Sidxram-cumback-lyrics
20 Now scraping Sidxram-yak

9 Now scraping Lsp-and-pharaoh-pornstar-lyrics
10 Now scraping Lsp-labyrinth-of-reflections-lyrics
11 Now scraping Lsp-spit-into-eternity-lyrics
12 Now scraping Lsp-moneys-not-a-problem-lyrics
13 Now scraping Lsp-end-of-the-world-lyrics
14 Now scraping Lsp-garbage-lyrics
15 Now scraping Lsp-ok-lyrics
16 Now scraping Lsp-and-pharaoh-goldfish-lyrics
17 Now scraping Lsp-resurrection-lyrics
18 Now scraping Lsp-poteryan-i-ne-nayden-lyrics
19 Now scraping Lsp-and-pharaoh-pit-lyrics
20 Now scraping Lsp-and-pharaoh-cake-lyrics
21 Now scraping Lsp-blue-lyrics
22 Now scraping Lsp-cocktail-lyrics
23 Now scraping Lsp-bitch-lyrics
24 Now scraping Lsp-levels-lyrics
25 Now scraping Lsp-vinaigrette-lyrics
26 Now scraping Lsp-better-than-internet-lyrics
27 Now scraping Lsp-and-pharaoh-9-9-lives-lyrics
28 Now scraping Lsp-let-them-talk-lyrics
29 Now scraping Lsp-biggie-lyrics
30 Now scraping Lsp-more-money-lyrics
31 Now scraping Lsp-vverh-vniz-lyrics
32 Now scraping Lsp-kanat-lyrics
33 Now scraping Lsp-

1 Now scraping Scriptonite-situation-lyrics
2 Now scraping Scriptonite-animals-lyrics
3 Now scraping Scriptonite-for-real-lyrics
4 Now scraping Scriptonite-butter-lyrics
5 Now scraping Scriptonite-napomni-lyrics
6 Now scraping Scriptonite-intro-time-to-come-back-lyrics
7 Now scraping Scriptonite-priton-lyrics
8 Now scraping Scriptonite-ostav-ih-lyrics
9 Now scraping Scriptonite-dont-need-to-recognize-me-lyrics
10 Now scraping Scriptonite-mirrors-lyrics
11 Now scraping Scriptonite-718-mr-718-lyrics
12 Now scraping Scriptonite-1000-lyrics
13 Now scraping Scriptonite-temno-lyrics
14 Now scraping Scriptonite-waste-of-time-lyrics
15 Now scraping 104-and-truwer-safari-lyrics
16 Now scraping Scriptonite-it-is-love-lyrics
17 Now scraping Scriptonite-party-lyrics
18 Now scraping Scriptonite-tantsuy-sama-lyrics
19 Now scraping Scriptonite-bro-lyrics
20 Now scraping Scriptonite-lokony-lyrics
21 Now scraping Scriptonite-boy-lyrics
22 Now scraping Scriptonite-interview-lyrics
23 Now scraping Script

...

74 Now scraping Lsp-and-pharaoh-9-9-lives-lyrics
75 Now scraping Pharaoh-you-on-the-moon-lyrics
76 Now scraping Pharaoh-and-i61-from-dusk-till-dawn-lyrics
77 Now scraping Pharaoh-and-i61-empire-strikes-back-lyrics
78 Now scraping Pharaoh-uadjet-lyrics
79 Now scraping Pharaoh-to-the-bottom-lyrics
80 Now scraping Pharaoh-exodus-lyrics
81 Now scraping Pharaoh-lycanthropy-lyrics
82 Now scraping Pharaoh-18-club-18-utterly-lyrics
83 Now scraping Pharaoh-fire-lyrics
84 Now scraping Pharaoh-telegram-snippet-1-lyrics
85 Now scraping Pharaoh-liquid-death-lyrics
86 Now scraping Pharaoh-phoenix-lyrics
87 Now scraping Pharaoh-new-era-lyrics
88 Now scraping Pharaoh-unreleased-track-3-lyrics
89 Now scraping Pharaoh-snippet-200617-lyrics
90 Now scraping Lil-morty-i-need-you-lyrics
Finding the ID of an artist макс корж
ID of an artist макс корж is 1050560
Getting the list of songs
Going through page 1
Без косяка (Bez kosyaka) by Макс Корж (Max Korzh) (Ft. Леха Лазько (Leha Lazko)) is a feat, skipping
G

Going through page 2
Going through page 3
0 Now scraping T-fest-one-thing-i-knew-exhalation-lyrics
1 Now scraping T-fest-fly-away-lyrics
2 Now scraping T-fest-dont-forget-lyrics
3 Now scraping T-fest-goodbye-lyrics
4 Now scraping T-fest-mom-allowed-lyrics
5 Now scraping T-fest-moon-lyrics
6 Now scraping T-fest-yo-yo-nope-lyrics
7 Now scraping T-fest-dirt-lyrics
8 Now scraping T-fest-not-for-me-lyrics
9 Now scraping T-fest-pray-for-yourself-lyrics
10 Now scraping T-fest-new-day-lyrics
11 Now scraping T-fest-violet-sky-lyrics
12 Now scraping T-fest-i-wont-give-up-lyrics
13 Now scraping T-fest-time-to-play-lyrics
14 Now scraping T-fest-youth-lyrics
15 Now scraping T-fest-beauty-lyrics
16 Now scraping T-fest-chemicals-lyrics
17 Now scraping T-fest-soul-singer-lyrics
18 Now scraping T-fest-past-the-houses-lyrics
19 Now scraping T-fest-hype-lyrics
20 Now scraping T-fest-has-melted-lyrics
21 Now scraping T-fest-blossom-lyrics
22 Now scraping T-fest-outro-lyrics
23 Now scraping T-fest-numbers-

Going through page 3
0 Now scraping Versus-battle-08-01-17-vs-lyrics
Finding the ID of an artist SD aka Sadist
ID of an artist SD aka Sadist is 1039745
Getting the list of songs
Going through page 1
Desiigner by Bumble Beezy (Ft. СД (SD aka Sadist)) is a feat, skipping
Гоша Рубчинский (Gosha Rubchinskiy) by Хан Замай (Khan Zamay) & Слава КПСС (Slava KPSS) (Ft. Букер (Booker), Хан Замай (Khan Zamay), ​монеточка (monetochka), Овсянкин (Ovsyankin), СД (SD aka Sadist), Слава КПСС (Slava KPSS) & Zoo In Space) is a feat, skipping
Operation Payback by Schokk (Ft. Oxxxymiron & СД (SD aka Sadist)) is a feat, skipping
Жесть он крут (He is so Cool) by СД (SD aka Sadist) (Ft. Букер (Booker), Bumble Beezy, Слава КПСС (Slava KPSS) & Young P&H) is a feat, skipping
Special Delivery by СД (SD aka Sadist) (Ft. Oxxxymiron) is a feat, skipping
Lola Taylor by Овсянкин (Ovsyankin) (Ft. Хан Замай (Khan Zamay), найтивыход (naitivihod), СД (SD aka Sadist), Слава КПСС (Slava KPSS), Stephan Pie, Unda Scope & You

1 Now scraping Oxxxymiron-last-call-lyrics
2 Now scraping Oxxxymiron-yeti-i-deti-lyrics
3 Now scraping Oxxxymiron-i-am-a-hater-lyrics
4 Now scraping Sd-aka-sadist-porn-lyrics
5 Now scraping Dyadya-zhenya-e-ochevidnoe-neveroyatnoe-lyrics
6 Now scraping Sd-aka-sadist-kischuk-and-maddyson-diss-lyrics
7 Now scraping Sd-and-rickey-f-google-lyrics
8 Now scraping Sd-aka-sadist-girl-is-waiting-lyrics
9 Now scraping Sd-aka-sadist-dont-think-about-sales-lyrics
10 Now scraping Sd-and-rickey-f-alpinist-lyrics
11 Now scraping Sd-aka-sadist-thousands-of-flows-lyrics
12 Now scraping Sd-aka-sadist-show-me-god-lyrics
13 Now scraping Sd-aka-sadist-l-love-people-lyrics
14 Now scraping Sd-aka-sadist-this-night-lyrics
15 Now scraping Sd-aka-sadist-aka-sadist-lyrics
16 Now scraping Sd-aka-sadist-hip-hopru-battle-7-round-8-lyrics
17 Now scraping Sd-aka-sadist-my-struggle-lyrics
18 Now scraping Sd-aka-sadist-you-can-hate-me-lyrics
19 Now scraping Sd-aka-sadist-why-this-world-of-evil-lyrics
20 Now scraping Sd-

20 Now scraping Face-night-fever-lyrics
21 Now scraping Face-where-you-are-not-lyrics
22 Now scraping Face-lash-bonus-track-lyrics
23 Now scraping Face-ten-pensions-lyrics
24 Now scraping Face-thousands-birds-lyrics
25 Now scraping Face-number-three-lyrics
26 Now scraping Face-vlone-lyrics
27 Now scraping Face-if-rap-does-not-give-lyrics
28 Now scraping Face-there-is-no-friendship-lyrics
29 Now scraping Face-take-whatever-you-want-lyrics
30 Now scraping Face-sub-zero-freestyle-lyrics
31 Now scraping Face-your-warm-bed-lyrics
32 Now scraping Face-in-the-rooms-of-darkness-lyrics
33 Now scraping Face-dick-pic-lyrics
34 Now scraping Face-psp-lyrics
35 Now scraping Face-blazer-lyrics
36 Now scraping Face-festival-of-colors-lyrics
37 Now scraping Face-skate-lyrics
38 Now scraping Face-suicide-lyrics
39 Now scraping Face-life-lyrics
40 Now scraping Face-kanji-lyrics
41 Now scraping Face-payback-lyrics
42 Now scraping Face-two-castles-lyrics
43 Now scraping Face-gold-on-my-neck-lyrics
44 Now s

In [ ]:
def get_percentage(words, total):
    return round(words*100/total, 2)

def ord_dict(input_dict):
    return OrderedDict(sorted(input_dict.items(), key = itemgetter(1), reverse = True))

class Lyric_file(object): 
    the_text = ''
    the_list = []
    the_counter = {}
    the_total = 0
    def __init__(self, location):
        self.location = location
        self.the_text = open(self.location, 'r').read()
        self.the_list = self.the_text.split()
        self.the_counter = Counter(self.the_list)
        self.the_total = len(self.the_list)
        
    def get_most_common(self, number):
        return self.the_counter.most_common(number)
    
    def get_most_common_percentage(self, number):
        result_list = []
        for word in self.get_most_common(number):
            result_list.append((word[0], get_percentage(word[1], len(self.get_list()))))
        return result_list
    
    def get_percentage_of_word(self, word):
        return get_percentage(self.the_counter[word],len(self.the_list))
    
    def get_vocab_size(self):
        return len(self.the_list)/len(self.the_counter)
    

In [10]:
list_tuple = {}
for artist in os.listdir('./lyrics/'):
    lyric = Lyric_file('./lyrics/' + artist)
    list_tuple[artist] = lyric.get_percentage_of_word("я")

pprint(ord_dict(list_tuple))

{'T-Fest_lyrics.txt': 5.22,
 'pharaoh_lyrics.txt': 4.97,
 'Bumble_Beezy_lyrics.txt': 4.14,
 'Тимати_lyrics.txt': 3.7,
 'Скриптонит_lyrics.txt': 3.28,
 'Хаски_lyrics.txt': 3.2,
 'ЛСП_lyrics.txt': 2.98,
 'guf_lyrics.txt': 2.91,
 'Эджей_lyrics.txt': 2.88,
 'Грязный_рамирес_lyrics.txt': 2.76,
 'макс_корж_lyrics.txt': 2.67,
 'Слава_КПСС_lyrics.txt': 2.65,
 'SD_aka_Sadist_lyrics.txt': 2.54,
 'oxxxymiron_lyrics.txt': 2.53,
 'Big_russian_boss_lyrics.txt': 2.46,
 'Птаха_lyrics.txt': 2.36,
 'rickey_f_lyrics.txt': 2.36,
 'Кровосток_lyrics.txt': 2.08,
 'Каста_lyrics.txt': 1.85,
 'ноггано_lyrics.txt': 1.67,
 'ATL_lyrics.txt': 0.0}


In [65]:
list_tuple = {}
for artist in os.listdir('./lyrics/'):
    lyric = Lyric_file('./lyrics/' + artist)
    list_tuple[artist] = lyric.get_vocab_size()
pprint(OrderedDict(sorted(list_tuple.items(), key = itemgetter(1), reverse = True)))
    

{'ATL_lyrics.txt': 10.422311827956989,
 'T-Fest_lyrics.txt': 4.238836967808931,
 'Скриптонит_lyrics.txt': 4.186169134049888,
 'pharaoh_lyrics.txt': 4.035280047837353,
 'Bumble_Beezy_lyrics.txt': 4.022329589971272,
 'ЛСП_lyrics.txt': 3.9103348358016743,
 'Тимати_lyrics.txt': 3.746849315068493,
 'guf_lyrics.txt': 3.7419499910055767,
 'SD_aka_Sadist_lyrics.txt': 3.517179487179487,
 'Эджей_lyrics.txt': 3.0998680158380996,
 'макс_корж_lyrics.txt': 3.083668543845535,
 'Каста_lyrics.txt': 3.0067528458421764,
 'Хаски_lyrics.txt': 2.8574816220061656,
 'oxxxymiron_lyrics.txt': 2.787449655710017,
 'Big_russian_boss_lyrics.txt': 2.757631578947368,
 'ноггано_lyrics.txt': 2.6236297198538368,
 'Кровосток_lyrics.txt': 2.4446957236842106,
 'Грязный_рамирес_lyrics.txt': 2.4247372675828616,
 'Слава_КПСС_lyrics.txt': 2.263233190271817,
 'Птаха_lyrics.txt': 2.1091617933723197,
 'rickey_f_lyrics.txt': 2.1091617933723197}


In [22]:
list_of_me = ['я', "мой", "моя", "мои", "меня", "мы", "наш"]
lyricobj = Lyric_file('./lyrics/guf_lyrics.txt')

In [23]:
total = 0
for me in list_of_me:
    total += lyricobj.the_counter[me]
print(total)
print(len(lyricobj.the_list))
print(len(lyricobj.the_list)/total)

1839
41603
22.6226209896683


In [25]:
with open('./something.txt', 'r+') as txt:
    if "Hallo" in txt.read():
        txt.write(txt.read() + "Somethign\n")
    else:
        txt.write(txt.read() + "Hllo\n")
    print(txt.read())
    